<a href="https://colab.research.google.com/github/ronyates47/health/blob/main/RBCA_analysis_202509.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Cell 1-credentials sets credentials and base info.

import os

os.environ['FTP_HOST'] = 'ftp.one-name.net'
os.environ['FTP_PORT'] = '21'
os.environ['FTP_USER'] = 'admin@yates.one-name.net'
os.environ['FTP_PASS'] = 'v(i83lfQB@dB'   # <-- rotate if you changed it

# Optional but helpful for printed URLs
os.environ['BASE_URL']  = 'https://yates.one-name.net'
os.environ['REMOTE_DIR'] = 'public_html'

print("FTP environment variables set for this runtime.")


FTP environment variables set for this runtime.


In [ ]:
# === Cell 2: RBCA pipeline (with forward-fill for missing metrics) ===
# - Mounts Drive and ensures /MyDrive/RBCA_v1 exists
# - Seeds rbca_data.csv if missing (BASE row + one sample row)
# - Forward-fills empty inputs from prior row (vo2/hrv/rhr/sys/dia/bmi/waist/hip)
# - Computes per-entry RBCA and weekly summaries
# - Writes rbca_results.(csv/html) and rbca_weekly.(csv/html)

import os, math, csv
import numpy as np
import pandas as pd
from datetime import date
from google.colab import drive

# 1) Mount Drive & set folder
drive.mount('/content/drive', force_remount=False)
FOLDER_NAME = "RBCA_v1"
BASE_DIR = f"/content/drive/MyDrive/{FOLDER_NAME}"
os.makedirs(BASE_DIR, exist_ok=True)

DATA_CSV  = os.path.join(BASE_DIR, "rbca_data.csv")
RES_CSV   = os.path.join(BASE_DIR, "rbca_results.csv")
RES_HTML  = os.path.join(BASE_DIR, "rbca_results.html")
WEEK_CSV  = os.path.join(BASE_DIR, "rbca_weekly.csv")
WEEK_HTML = os.path.join(BASE_DIR, "rbca_weekly.html")

# 2) Seed starter CSV if missing (32 columns total, BASE row first)
header = [
    "date","vo2","hrv","rhr","sys","dia","bmi","waist","hip",
    "age0","k",
    "mu_vo2","mu_hrv","mu_rhr","mu_sys","mu_dia","mu_bmi","mu_whr",
    "sigma_vo2","sigma_hrv","sigma_rhr","sigma_sys","sigma_dia","sigma_bmi","sigma_whr",
    "w_vo2","w_hrv","w_rhr","w_sys","w_dia","w_bmi","w_whr"
]
if not os.path.exists(DATA_CSV):
    base_row = [
        "BASE",
        19.0, 34, 52, 102, 67, 25.2, 41, 40,     # vo2..hip (your baseline measurements)
        78, 5.0,                                  # age0, k
        19.0, 34, 52, 102, 67, 25.2, 1.025,       # mu_*
        0.95, 3.0, 3.0, 5.1, 3.35, 1.26, 0.02,    # sigma_*
        0.30, 0.25, 0.15, 0.10, 0.05, 0.10, 0.05  # weights (sum to 1)
    ]
    sample_row = [date.today().isoformat(), 22.4, 34, 52, 102, 67, 25.2, 41, 40] + [""]*23

    with open(DATA_CSV, "w", newline="") as f:
        w = csv.writer(f)
        w.writerow(header)
        w.writerow(base_row)
        w.writerow(sample_row)
    print(f"✅ Created starter {DATA_CSV}")
else:
    print(f"✅ Found existing {DATA_CSV}")

# 3) Helpers
def safe_div(a, b):
    try:
        return a / b if (a is not None and b not in (None, 0)) else np.nan
    except Exception:
        return np.nan

def log_step_z(x, mu, step=0.05):
    """z = ln(x/mu) / ln(1+step)."""
    try:
        if x in (None, "") or mu in (None, ""): return np.nan
        x = float(x); mu = float(mu)
        if x <= 0 or mu <= 0: return np.nan
        denom = math.log(1.0 + step)
        return math.log(x / mu) / denom if denom != 0 else np.nan
    except Exception:
        return np.nan

def linear_good_is_up(x, mu, sigma):
    """z = (mu - x)/sigma (so higher x → worse → negative z)."""
    try:
        if x in (None, ""): return np.nan
        x = float(x); mu = float(mu); sigma = float(sigma)
        if sigma == 0: return np.nan
        return (mu - x) / sigma
    except Exception:
        return np.nan

def parse_float_required(val, name):
    val = (val or "").strip()
    if val == "":
        raise ValueError(f"Missing BASE value for '{name}'.")
    try:
        return float(val)
    except:
        raise ValueError(f"BASE value for '{name}' must be numeric; got '{val}'.")

def to_float_series(s):
    return pd.to_numeric(s.replace("", np.nan), errors="coerce")

# 4) Load CSV and parse BASE
df = pd.read_csv(DATA_CSV, dtype=str).fillna("")
expected_cols = header
missing = [c for c in expected_cols if c not in df.columns]
if missing:
    raise ValueError(f"{DATA_CSV} missing required columns: {missing}")

if df.empty:
    raise ValueError("rbca_data.csv is empty. Need a BASE row and at least one data row.")

base = df.iloc[0].copy()

age0 = parse_float_required(base["age0"], "age0")
k    = parse_float_required(base["k"], "k")

mu_vo2 = parse_float_required(base["mu_vo2"], "mu_vo2")
mu_hrv = parse_float_required(base["mu_hrv"], "mu_hrv")
mu_rhr = parse_float_required(base["mu_rhr"], "mu_rhr")
mu_sys = parse_float_required(base["mu_sys"], "mu_sys")
mu_dia = parse_float_required(base["mu_dia"], "mu_dia")
mu_bmi = parse_float_required(base["mu_bmi"], "mu_bmi")
mu_whr = parse_float_required(base["mu_whr"], "mu_whr")

sigma_vo2 = parse_float_required(base["sigma_vo2"], "sigma_vo2")
sigma_hrv = parse_float_required(base["sigma_hrv"], "sigma_hrv")
sigma_rhr = parse_float_required(base["sigma_rhr"], "sigma_rhr")
sigma_sys = parse_float_required(base["sigma_sys"], "sigma_sys")
sigma_dia = parse_float_required(base["sigma_dia"], "sigma_dia")
sigma_bmi = parse_float_required(base["sigma_bmi"], "sigma_bmi")
sigma_whr = parse_float_required(base["sigma_whr"], "sigma_whr")

w_vo2 = parse_float_required(base["w_vo2"], "w_vo2")
w_hrv = parse_float_required(base["w_hrv"], "w_hrv")
w_rhr = parse_float_required(base["w_rhr"], "w_rhr")
w_sys = parse_float_required(base["w_sys"], "w_sys")
w_dia = parse_float_required(base["w_dia"], "w_dia")
w_bmi = parse_float_required(base["w_bmi"], "w_bmi")
w_whr = parse_float_required(base["w_whr"], "w_whr")

w_sum = w_vo2 + w_hrv + w_rhr + w_sys + w_dia + w_bmi + w_whr
if not np.isclose(w_sum, 1.0, atol=1e-6):
    raise ValueError(f"Weights must sum to 1.0; got {w_sum:.6f}")

# 5) Prepare data rows
data = df.iloc[1:].copy()
# drop fully blank rows (all primary inputs empty)
data = data[~(data[["date","vo2","hrv","rhr","sys","dia","bmi","waist","hip"]].eq("").all(axis=1))]
if data.empty:
    print("No data rows yet (only BASE). Creating empty outputs.")
    pd.DataFrame().to_csv(RES_CSV, index=False)
    with open(RES_HTML, "w") as f: f.write("<p>No results yet.</p>")
    pd.DataFrame().to_csv(WEEK_CSV, index=False)
    with open(WEEK_HTML, "w") as f: f.write("<p>No weekly results yet.</p>")
    raise SystemExit

# parse date column FIRST (so fill-forward happens in true time order)
data["date_parsed"] = pd.to_datetime(data["date"].str.strip(), errors="coerce", utc=False)

# numeric parse for metrics
metrics = ["vo2","hrv","rhr","sys","dia","bmi","waist","hip"]
for m in metrics:
    data[m] = to_float_series(data[m])

# === Forward-fill missing numeric inputs from prior rows ===
# sort by date for logical carry-forward
data = data.sort_values("date_parsed", na_position="last").reset_index(drop=True)
data[metrics] = data[metrics].ffill()

# derived metric after fill
data["whr"] = data.apply(lambda r: safe_div(r["waist"], r["hip"]), axis=1)

# 6) z-scores
data["z_vo2"] = data["vo2"].apply(lambda x: log_step_z(x, mu_vo2, step=0.05))
data["z_hrv"] = data["hrv"].apply(lambda x: log_step_z(x, mu_hrv, step=0.05))
data["z_rhr"] = data["rhr"].apply(lambda x: linear_good_is_up(x, mu_rhr, sigma_rhr))
data["z_sys"] = data["sys"].apply(lambda x: linear_good_is_up(x, mu_sys, sigma_sys))
data["z_dia"] = data["dia"].apply(lambda x: linear_good_is_up(x, mu_dia, sigma_dia))
data["z_bmi"] = data["bmi"].apply(lambda x: linear_good_is_up(x, mu_bmi, sigma_bmi))
data["z_whr"] = data["whr"].apply(lambda x: linear_good_is_up(x, mu_whr, sigma_whr))

# 7) composite, RBCA, and contributions
data["C"] = (
    w_vo2 * data["z_vo2"] +
    w_hrv * data["z_hrv"] +
    w_rhr * data["z_rhr"] +
    w_sys * data["z_sys"] +
    w_dia * data["z_dia"] +
    w_bmi * data["z_bmi"] +
    w_whr * data["z_whr"]
)
data["RBCA"] = age0 - k * data["C"]

data["yrs_vo2"] = k * w_vo2 * data["z_vo2"]
data["yrs_hrv"] = k * w_hrv * data["z_hrv"]
data["yrs_rhr"] = k * w_rhr * data["z_rhr"]
data["yrs_sys"] = k * w_sys * data["z_sys"]
data["yrs_dia"] = k * w_dia * data["z_dia"]
data["yrs_bmi"] = k * w_bmi * data["z_bmi"]
data["yrs_whr"] = k * w_whr * data["z_whr"]
data["yrs_total_delta"] = (age0 - data["RBCA"])

# 8) order, round, save per-entry results
cols_out = [
    "date","date_parsed","vo2","hrv","rhr","sys","dia","bmi","waist","hip","whr",
    "z_vo2","z_hrv","z_rhr","z_sys","z_dia","z_bmi","z_whr",
    "C","RBCA",
    "yrs_vo2","yrs_hrv","yrs_rhr","yrs_sys","yrs_dia","yrs_bmi","yrs_whr","yrs_total_delta"
]
res = data[cols_out].sort_values("date_parsed", na_position="last").reset_index(drop=True)

round_map = {
    "vo2":1, "hrv":1, "rhr":1, "sys":0, "dia":0, "bmi":1,
    "whr":3, "C":3, "RBCA":2,
    "z_vo2":3,"z_hrv":3,"z_rhr":3,"z_sys":3,"z_dia":3,"z_bmi":3,"z_whr":3,
    "yrs_vo2":2,"yrs_hrv":2,"yrs_rhr":2,"yrs_sys":2,"yrs_dia":2,"yrs_bmi":2,"yrs_whr":2,"yrs_total_delta":2
}
for col, nd in round_map.items():
    if col in res.columns:
        res[col] = res[col].astype(float).round(nd)

res.to_csv(RES_CSV, index=False)
res.to_html(RES_HTML, index=False, justify="center")

# === One-and-done narrative: What Changed + HRV Assessment (append once) ===

def _fmt(x, nd=1):
    try:
        return f"{float(x):.{nd}f}"
    except Exception:
        return "—"

def describe_change(name, old, new, unit="", thresh=0):
    if pd.isna(old) or pd.isna(new):
        return None
    delta = float(new) - float(old)
    if abs(delta) <= thresh:
        return None
    direction = "rose" if delta > 0 else "fell"
    return f"{name} {direction} from {_fmt(old)} → {_fmt(new)}{unit} ({delta:+.{1 if unit else 1}f})"

# Pick latest and previous rows (after sorting above)
narrative_html = ""
if len(res) >= 2:
    latest = res.iloc[-1]
    prev   = res.iloc[-2]

    # Build change bullets (tune thresholds if you like)
    changes = [
        describe_change("VO₂-max",    prev["vo2"],  latest["vo2"],  " ml/kg/min", 0.5),
        describe_change("HRV",         prev["hrv"], latest["hrv"], " ms",        2),
        describe_change("Resting HR",  prev["rhr"], latest["rhr"], " bpm",       1),
        describe_change("Systolic BP", prev["sys"], latest["sys"], " mmHg",      1),
        describe_change("Diastolic BP",prev["dia"], latest["dia"], " mmHg",      1),
        describe_change("BMI",         prev["bmi"], latest["bmi"], "",           0.1),
        describe_change("Waist",       prev["waist"], latest["waist"], " cm",    1),
        describe_change("Hip",         prev["hip"], latest["hip"], " cm",        1),
    ]
    changes = [c for c in changes if c]

    if changes:
        what_changed = (
            f"<h3>What Changed</h3>"
            f"<p>Compared with {prev['date']}, the latest RBCA ({_fmt(latest['RBCA'], 1)}) reflects: "
            + "; ".join(changes) + ".</p>"
        )
    else:
        what_changed = (
            f"<h3>What Changed</h3>"
            f"<p>No meaningful metric changes versus {prev['date']}.</p>"
        )
else:
    latest = res.iloc[-1]  # still safe
    what_changed = "<h3>What Changed</h3><p>Only one data row—no comparison yet.</p>"

# HRV assessment based on latest HRV
def hrv_assessment(hrv):
    if pd.isna(hrv):
        return "HRV data is unavailable."
    hrv = float(hrv)
    if hrv < 30:
        return (f"Your HRV of {_fmt(hrv)} ms is on the low side for your age, "
                "which can reflect stress load or incomplete recovery. Consider sleep, hydration, and easy days.")
    elif 30 <= hrv < 40:
        return (f"Your HRV of {_fmt(hrv)} ms is within the typical range for your age, "
                "suggesting moderate parasympathetic tone and recovery.")
    elif 40 <= hrv < 50:
        return (f"Your HRV of {_fmt(hrv)} ms is above average for your age, "
                "indicating good cardiovascular fitness and recovery.")
    else:
        return (f"Your HRV of {_fmt(hrv)} ms is excellent for your age, "
                "suggesting high autonomic adaptability and recovery capacity.")

hrv_block = f"<h3>HRV Assessment</h3><p>{hrv_assessment(latest['hrv'])}</p>"

# Wrap and append once
narrative_html = (
    "<div id='rbca-narrative' style='margin-top:18px;'>"
    + what_changed
    + hrv_block
    + "</div>"
)

with open(RES_HTML, "a", encoding="utf-8") as f:
    f.write("\n<br><br>\n" + narrative_html)


print("✅ RBCA results written to:")
print("  ", RES_CSV)
print("  ", RES_HTML)
print("  ", WEEK_CSV)
print("  ", WEEK_HTML)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Found existing /content/drive/MyDrive/RBCA_v1/rbca_data.csv
✅ RBCA results written to:
   /content/drive/MyDrive/RBCA_v1/rbca_results.csv
   /content/drive/MyDrive/RBCA_v1/rbca_results.html
   /content/drive/MyDrive/RBCA_v1/rbca_weekly.csv
   /content/drive/MyDrive/RBCA_v1/rbca_weekly.html


In [ ]:
# === Cell 3 (account-root) — Upload RBCA + wrapper with BIG "Latest Cardio Age" banner ===
# Prereqs: run Cell 1 (env vars) and Cell 2 (generate RBCA outputs) first.

import os, io
import pandas as pd
from ftplib import FTP_TLS
from datetime import datetime
from zoneinfo import ZoneInfo
from google.colab import drive

# 1) Mount Drive
drive.mount('/content/drive', force_remount=False)

# 2) Paths & env
FOLDER_NAME = "RBCA_v1"
BASE_DIR = f"/content/drive/MyDrive/{FOLDER_NAME}"
RES_CSV   = os.path.join(BASE_DIR, "rbca_results.csv")

HOST = os.environ.get("FTP_HOST")
PORT = int(os.environ.get("FTP_PORT", "21"))
USER = os.environ.get("FTP_USER")
PASS = os.environ.get("FTP_PASS")
BASE_URL = os.environ.get("BASE_URL", "https://yates.one-name.net")
assert HOST and USER and PASS, "Missing FTP_HOST / FTP_USER / FTP_PASS (run Cell 1)."

REMOTE_DIR = ""                       # <-- LIVE docroot for your domain
WRAPPER_REMOTE = "yates_health_metrics.htm"

LOCAL_FILES = [
    ("rbca_results.html", "rbca_results.html"),
    ("rbca_results.csv",  "rbca_results.csv"),
    ("rbca_weekly.html",  "rbca_weekly.html"),
    ("rbca_weekly.csv",   "rbca_weekly.csv"),
]

# 3) Read latest RBCA (for the banner)
latest_cardio_age_display = "N/A"
try:
    df = pd.read_csv(RES_CSV)
    if "date_parsed" in df.columns and "RBCA" in df.columns and not df.empty:
        # sort by date_parsed if present; else just take last row
        if pd.api.types.is_datetime64_any_dtype(df["date_parsed"]):
            d = df.copy()
        else:
            d = df.copy()
            d["date_parsed"] = pd.to_datetime(d["date_parsed"], errors="coerce")
        d = d.dropna(subset=["RBCA"])
        if not d.empty:
            d = d.sort_values("date_parsed", na_position="last")
            latest = pd.to_numeric(d["RBCA"], errors="coerce").dropna()
            if not latest.empty:
                # format as integer (e.g., 73). Change to one decimal if you prefer: f"{latest.iloc[-1]:.1f}"
                latest_cardio_age_display = f"{int(round(latest.iloc[-1]))}"
except Exception as e:
    # Keep N/A if anything goes wrong; still proceed with upload
    print("Note: Could not compute latest RBCA for banner:", repr(e))

# 4) Build wrapper with cache-busting + BIG banner
now = datetime.now(ZoneInfo("America/New_York"))
stamp = now.strftime("%Y%m%d-%H%M%S")
try:
    updated_str = now.strftime("%d %B %Y at %-I:%M %p EDT")
except:
    updated_str = now.strftime("%d %B %Y at %I:%M %p EDT").replace(" 0"," ")

def vpath(name):  # root path with cache-busting
    return f"/{name}?v={stamp}"

WRAPPER_HTML = f"""<!DOCTYPE html>
<html lang="en">
<head>
  <meta charset="UTF-8" />
  <meta name="viewport" content="width=device-width, initial-scale=1" />
  <title>Yates Health Metrics</title>
  <!-- RBCA WRAPPER DEPLOY {stamp} -->
  <meta http-equiv="Cache-Control" content="no-cache, no-store, must-revalidate" />
  <meta http-equiv="Pragma" content="no-cache" />
  <meta http-equiv="Expires" content="0" />
  <style>
    body {{ font-family: Arial, Helvetica, sans-serif; margin: 0; padding: 0; background: #f7f7f7; }}
    header {{ background: #002b45; color: #fff; padding: 16px 20px; }}
    header h1 {{ margin: 0; font-size: 20px; }}
    header .meta {{ font-size: 13px; opacity: 0.85; margin-top: 6px; }}
    .banner {{
      margin: 12px 0 0;
      font-size: 34px; font-weight: 800; line-height: 1.2;
      color: #fffb;  /* slightly translucent white for style */
      text-shadow: 0 1px 2px rgba(0,0,0,0.25);
    }}
    .banner .value {{
      color: #ffea00; /* bright highlight for the age number */
      font-size: 40px;
    }}
    main {{ padding: 16px 12px 32px; max-width: 1200px; margin: 0 auto; }}
    .card {{ background: #fff; border: 1px solid #ddd; border-radius: 8px; padding: 12px; margin-bottom: 16px; }}
    .card h2 {{ margin: 6px 0 12px; font-size: 18px; color: #222; }}
    .links a {{ margin-right: 12px; font-size: 14px; }}
    iframe {{ width: 100%; height: 640px; border: 0; background: #fff; }}
    @media (max-width: 700px) {{
      iframe {{ height: 520px; }}
      .banner {{ font-size: 28px; }}
      .banner .value {{ font-size: 34px; }}
    }}
  </style>
</head>
<body>
  <header>
    <h1>Yates Health Metrics</h1>
    <div class="meta">Updated: {updated_str}</div>
    <div class="banner">Latest Cardio Age is <span class="value">{latest_cardio_age_display}</span></div>
  </header>
  <main>
    <div class="card">
      <h2>BioCardio Age — Daily/Entry Results</h2>
      <div class="links">
        <a href="{vpath('rbca_results.html')}" target="_blank">Open in new tab</a>
        <a href="{vpath('rbca_results.csv')}"  target="_blank">Download CSV</a>
      </div>
      <iframe src="{vpath('rbca_results.html')}" title="RBCA Results"></iframe>
    </div>

    <div class="card">
      <h2>BioCardio Age — Weekly Summary</h2>
      <div class="links">
        <a href="{vpath('rbca_weekly.html')}" target="_blank">Open in new tab</a>
        <a href="{vpath('rbca_weekly.csv')}"  target="_blank">Download CSV</a>
      </div>
      <iframe src="{vpath('rbca_weekly.html')}" title="RBCA Weekly"></iframe>
    </div>
  </main>
</body>
</html>"""

# 5) FTP helpers
def cd_path(ftps, path: str):
    """cd into path; empty string = account root (do nothing)."""
    if not path: return
    for seg in path.replace("\\","/").split("/"):
        if seg:
            try: ftps.cwd(seg)
            except Exception:
                ftps.mkd(seg); ftps.cwd(seg)

def upload_file(ftps: FTP_TLS, local_path: str, remote_name: str):
    assert os.path.exists(local_path), f"Missing local file: {local_path}"
    with open(local_path, "rb") as fh:
        try: ftps.delete(remote_name)
        except Exception: pass
        ftps.storbinary(f"STOR {remote_name}", fh)
        try: ftps.sendcmd(f"SITE CHMOD 644 {remote_name}")
        except Exception: pass

# 6) Connect & upload
print(f"Connecting to {HOST}:{PORT} …")
with FTP_TLS() as ftps:
    ftps.connect(HOST, PORT, timeout=30)
    try: ftps.auth()
    except: pass
    ftps.login(USER, PASS)
    try: ftps.prot_p()
    except: pass
    ftps.set_pasv(True)

    # account root
    cd_path(ftps, REMOTE_DIR)  # empty => noop

    # upload artifacts
    live_urls = []
    for local_name, remote_name in LOCAL_FILES:
        path_local = os.path.join(BASE_DIR, local_name)
        print(f"Uploading {local_name} -> /{remote_name}")
        upload_file(ftps, path_local, remote_name)
        live_urls.append(f"{BASE_URL}/{remote_name}?v={stamp}")

    # upload wrapper
    print(f"Uploading wrapper -> /{WRAPPER_REMOTE}")
    try: ftps.delete(WRAPPER_REMOTE)
    except Exception: pass
    ftps.storbinary(f"STOR {WRAPPER_REMOTE}", io.BytesIO(WRAPPER_HTML.encode("utf-8")))
    try: ftps.sendcmd(f"SITE CHMOD 644 {WRAPPER_REMOTE}")
    except Exception: pass

print("\n✅ Live URLs:")
for u in live_urls:
    print("  ", u)
print("  ", f"{BASE_URL}/{WRAPPER_REMOTE}?v={stamp}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Connecting to ftp.one-name.net:21 …
Uploading rbca_results.html -> /rbca_results.html
Uploading rbca_results.csv -> /rbca_results.csv
Uploading rbca_weekly.html -> /rbca_weekly.html
Uploading rbca_weekly.csv -> /rbca_weekly.csv
Uploading wrapper -> /yates_health_metrics.htm

✅ Live URLs:
   https://yates.one-name.net/rbca_results.html?v=20250919-091726
   https://yates.one-name.net/rbca_results.csv?v=20250919-091726
   https://yates.one-name.net/rbca_weekly.html?v=20250919-091726
   https://yates.one-name.net/rbca_weekly.csv?v=20250919-091726
   https://yates.one-name.net/yates_health_metrics.htm?v=20250919-091726


# New Section